In [ ]:
!pip install pandas
!pip install matplotlib

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/home/fahd/Documents/headline-trader/headline-trader/src/training/full_df.csv')
df.head()

,timestamp,title,price,sentiment,sentiment_score,future_price,label,symbol,hour_of_day,day_of_week
0,2025-04-13 05:10:34.328105+00:00,Bitcoin’s first Ponzi scheme would be worth $1...,84593.72,neutral,0.936433,84488.16,hold,BTCUSDT,5,6
1,2025-04-13 05:13:56.575712+00:00,Bitcoin’s first Ponzi scheme would be worth $1...,84488.16,neutral,0.936433,84488.16,hold,BTCUSDT,5,6
2,2025-04-13 05:13:58.043529+00:00,SEC Signals Massive Crypto Trading Shakeup at ...,84488.16,negative,0.516813,84488.16,hold,BTCUSDT,5,6
3,2025-04-13 05:13:59.634376+00:00,Bitcoin Price Volatility Far Lower Than During...,84488.16,neutral,0.877506,84488.16,hold,BTCUSDT,5,6
4,2025-04-13 05:14:01.814910+00:00,Lomond boarding school to accept tuition fee p...,84488.16,neutral,0.880280,84488.16,hold,BTCUSDT,5,6


In [3]:
label_map = {'buy': 0, 'hold':1, 'sell': 2}
df['label'] = df['label'].map(label_map)
df['label']

0       1
1       1
2       1
3       1
4       1
       ..
3117    1
3118    1
3119    1
3120    1
3121    1
Name: label, Length: 3122, dtype: int64

In [7]:
features = df[['sentiment_score', 'price', 'hour_of_day', 'day_of_week']]
labels = df['label']

features, labels

(      sentiment_score      price  hour_of_day  day_of_week
 0            0.936433  84593.720            5            6
 1            0.936433  84488.160            5            6
 2            0.516813  84488.160            5            6
 3            0.877506  84488.160            5            6
 4            0.880280  84488.160            5            6
 ...               ...        ...          ...          ...
 3117         0.650255      2.081           15            5
 3118         0.615485      2.081           15            5
 3119         0.701910      2.081           15            5
 3120         0.934092      2.081           15            5
 3121         0.916843      2.081           15            5
 
 [3122 rows x 4 columns],
 0       1
 1       1
 2       1
 3       1
 4       1
        ..
 3117    1
 3118    1
 3119    1
 3120    1
 3121    1
 Name: label, Length: 3122, dtype: int64)

In [8]:
# convert to pytorch tensors

x = torch.tensor(features.values, dtype=torch.float32)
y = torch.tensor(labels.values, dtype=torch.long)

x, y

(tensor([[9.3643e-01, 8.4594e+04, 5.0000e+00, 6.0000e+00],
         [9.3643e-01, 8.4488e+04, 5.0000e+00, 6.0000e+00],
         [5.1681e-01, 8.4488e+04, 5.0000e+00, 6.0000e+00],
         ...,
         [7.0191e-01, 2.0810e+00, 1.5000e+01, 5.0000e+00],
         [9.3409e-01, 2.0810e+00, 1.5000e+01, 5.0000e+00],
         [9.1684e-01, 2.0810e+00, 1.5000e+01, 5.0000e+00]]),
 tensor([1, 1, 1,  ..., 1, 1, 1]))

In [10]:
# Training and testing

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)



In [11]:
# Using DataLoader for batching 
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [13]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)  # 3 classes: buy/sell/hold

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = MyModel(input_size=x.shape[1])

In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20
for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/20, Loss: 0.0000
Epoch 2/20, Loss: 0.0001
Epoch 3/20, Loss: 0.0000
Epoch 4/20, Loss: 0.0000
Epoch 5/20, Loss: 0.0000
Epoch 6/20, Loss: 0.0000
Epoch 7/20, Loss: 0.0000
Epoch 8/20, Loss: 0.0000
Epoch 9/20, Loss: 0.0000
Epoch 10/20, Loss: 0.0000
Epoch 11/20, Loss: 0.0000
Epoch 12/20, Loss: 0.0003
Epoch 13/20, Loss: 0.0000
Epoch 14/20, Loss: 0.0000
Epoch 15/20, Loss: 0.0002
Epoch 16/20, Loss: 0.0013
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0001
Epoch 20/20, Loss: 0.0000
